In [38]:
import os 
from typing import Any, Dict 

import google.generativeai as genai
import requests 
from dotenv import load_dotenv
import yfinance as yf

print("We are up and running!")

We are up and running!


In [ ]:
# Load environment variables from .env file
load_dotenv()

# Get the Gemini API key from environment variables
gemini_api_key = os.getenv("GEMINI_API_KEY")
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set or is set to a placeholder value. Please set it in your .env file.")

# Configure the Google Generative AI client with the API key
genai.configure(api_key=gemini_api_key)
# Load the Gemini model 
model = genai.GenerativeModel("gemini-2.0-flash")

print(f"Gemini model loaded successfully: {model.model_name}")

Gemini model loaded successfully: models/gemini-2.0-flash


In [ ]:
PROMPT = "What is the current price of Bitcoin?"
chat = model.start_chat()
response = chat.send_message(PROMPT)
print(f"Response from Gemini model: {response.text}")

Response from Gemini model: Unfortunately, I cannot give you the exact, real-time price of Bitcoin right now. The price of Bitcoin fluctuates very rapidly, even within seconds.

**However, I can tell you where to find the most up-to-date price:**

*   **Major Cryptocurrency Exchanges:** These are the most reliable sources. Some popular exchanges include:
    *   Coinbase
    *   Binance
    *   Kraken
    *   Gemini

*   **Financial Websites:** Many reputable financial websites track Bitcoin prices:
    *   Google Finance (search "Bitcoin price")
    *   Yahoo Finance (search "BTC-USD")
    *   Bloomberg
    *   MarketWatch

When you check, look for the "BTC/USD" or "XBT/USD" ticker, which represents the price of Bitcoin in US Dollars.



In [6]:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Unfortunately, I cannot give you the exact, real-time price of Bitcoin right now. The price of Bitcoin fluctuates very rapidly, even within seconds.\n\n**However, I can tell you where to find the most up-to-date price:**\n\n*   **Major Cryptocurrency Exchanges:** These are the most reliable sources. Some popular exchanges include:\n    *   Coinbase\n    *   Binance\n    *   Kraken\n    *   Gemini\n\n*   **Financial Websites:** Many reputable financial websites track Bitcoin prices:\n    *   Google Finance (search \"Bitcoin price\")\n    *   Yahoo Finance (search \"BTC-USD\")\n    *   Bloomberg\n    *   MarketWatch\n\nWhen you check, look for the \"BTC/USD\" or \"XBT/USD\" ticker, which represents the price of Bitcoin in US Dollars.\n"
              }
       

In [39]:
# url = f"https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT"
# # url = f"https://data.binance.com/api/v3/ticker?symbol=BTCUSDT"
# # url = f"https://testnet.binancefuture.com/fapi/v1/klines?symbol=BTCUSDT&interval=1h&limit=1"
# response = requests.get(url)
# data = response.json()
# print(data)

# Using CoinGecko API as an alternative
# url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd"
# response = requests.get(url)
# data = response.json()
# print(data)  # Print the current price of Bitcoin in USD

stock = yf.Ticker("BTC-USD")
price = stock.history(period="1d")["Close"].iloc[-1]
print(f"The current price of Bitcoin is: {price} USD")

The current price of Bitcoin is: 117734.75 USD


In [48]:
def get_crypto_price(symbol: str) -> Dict[str, Any]:
    """
    Fetch the current price of a cryptocurrency using CoinGecko API.
    
    Args:
        symbol (str): The symbol of the cryptocurrency (e.g., 'bitcoin').
    
    Returns:
        Dict[str, Any]: A dictionary containing the current price.
    """
    try:
        stock = yf.Ticker(symbol)
        price = stock.history(period="1d")["Close"].iloc[-1]
        return {symbol: {"usd": price}}
    except Exception as e:
        print(f"Error fetching price for {symbol}: {e}")
        return {symbol: {"usd": None}}

In [50]:
symbol = "BTC-USD"
price = get_crypto_price(symbol=symbol)
if price is not None:
    print(f"The current price of {symbol} is: {price[symbol]['usd']} USD")

The current price of BTC-USD is: 117732.265625 USD


In [51]:
tools = [
    {
        "function_declarations": [
            {
                "name": "get_crypto_price",
                "description": "Fetch the current price of a cryptocurrency using CoinGecko API.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "symbol": {
                            "type": "string",
                            "description": "The cryptocurrency trading symbol (e.g., 'BTC-USD', 'ETH-USD'). \
                                            The symbol for Bitcoin is 'BTC-USD'. \
                                            The symbol for Ethereum is 'ETH-USD'."
                        }
                    },
                    "required": ["symbol"]
                }
            }
        ]
    }
]

In [52]:
PROMPT = "What is the current price of Bitcoin?"
chat = model.start_chat()
response = chat.send_message(PROMPT, tools=tools)
print(f"Response from Gemini model: {response}")

Response from Gemini model: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "get_crypto_price",
                  "args": {
                    "symbol": "BTC-USD"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": 2.5517349260755713e-07
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 72,
        "candidates_token_count": 9,
        "total_token_count": 81
      },
      "model_version": "gemini-2.0-flash"
    }),
)


In [53]:
price = get_crypto_price(symbol="BTC-USD")

In [54]:
final_response = chat.send_message(str(price))
print(f"Final response from Gemini model: {final_response}")

Final response from Gemini model: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The current price of Bitcoin (BTC-USD) is $117758.5625 USD.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.12670340904822716
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 43,
        "candidates_token_count": 26,
        "total_token_count": 69
      },
      "model_version": "gemini-2.0-flash"
    }),
)


In [55]:
print(final_response.text)

The current price of Bitcoin (BTC-USD) is $117758.5625 USD.

